In [9]:
#Variables with global variables intializer and session 
import tensorflow as tf

x=tf.Variable(7,name="X")
y=tf.Variable(6,name="Y")
z=x*x+y
f1=z*2
f2=z*2
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    z_val=z.eval()
    f1_val=f1.eval()
    f2_val=f2.eval()
    #Better way of evaluating the above statements requires half computation as the same graph is used in both
    f1_val,f2_val=sess.run([f1,f2])
    print(z_val,f1_val,f2_val)

55 110 110


In [4]:
#Using a different graph
tf.reset_default_graph()

g=tf.Graph()

with g.as_default():
    x2=tf.Variable(60,name="X2")

x2.graph is g

True

In [18]:
#Linear Regression using formula ((xt.x)^-1).x.y)

tf.reset_default_graph()

import numpy as np
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()
m,n=housing.data.shape

housing_data_plus_bias=np.c_[housing.data,np.ones((m,1))] #adding bias feature as 1 for all entities

x=tf.constant(housing.data,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="Y")#reshaping as the actual variable 
                                                                    #shape comes as (20640,)

xt=tf.transpose(x)

theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt,x)),xt),y)

with tf.Session() as sess:
    theta_val=theta.eval()
    print(theta_val)

[[ 5.1351172e-01]
 [ 1.5650615e-02]
 [-1.8251595e-01]
 [ 8.6510789e-01]
 [ 7.7918567e-06]
 [-4.6992572e-03]
 [-6.3955665e-02]
 [-1.6385417e-02]]


In [34]:
#Using Gradient Descent with direct differential function and gradients function of tensorflow
tf.reset_default_graph()

import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

learning_rate=0.01
n_epochs=1000

housing=fetch_california_housing()
m,n=housing.data.shape

housing_data_plus_bias=np.c_[housing.data,np.ones((m,1))] #adding bias feature as 1 for all entities

scaler=StandardScaler()
housing_data_scaled=scaler.fit_transform(housing_data_plus_bias)

x=tf.constant(housing_data_scaled,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="Y")#reshaping as the actual variable 
                                                                    #shape comes as (20640,)
    
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),dtype=tf.float32,name="theta")
y_pred=tf.matmul(x,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
gradient=2/m * tf.matmul(tf.transpose(x),error)
#gradient=tf.gradients(mse,[theta])[0] #Gives the gradient values directly
training_op=tf.assign(theta,theta-learning_rate*gradient)
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op)
    print(theta.eval())
    print(gradient[0].get_shape())

[[ 8.5208672e-01]
 [ 1.3115743e-01]
 [-2.9265052e-01]
 [ 3.2138538e-01]
 [-2.8180477e-04]
 [-4.0854704e-02]
 [-7.8835082e-01]
 [-7.6087976e-01]
 [-8.8462591e-01]]
(1,)


In [36]:
#Using Gradient Descent with optimizers which does the differentiation and updates the weights by itself.
tf.reset_default_graph()

import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

learning_rate=0.01
n_epochs=1000

housing=fetch_california_housing()
m,n=housing.data.shape

housing_data_plus_bias=np.c_[housing.data,np.ones((m,1))] #adding bias feature as 1 for all entities

scaler=StandardScaler()
housing_data_scaled=scaler.fit_transform(housing_data_plus_bias)

x=tf.constant(housing_data_scaled,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="Y")#reshaping as the actual variable 
                                                                    #shape comes as (20640,)
    
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),dtype=tf.float32,name="theta")
y_pred=tf.matmul(x,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op)
    print(theta.eval())

[[ 0.84918886]
 [ 0.13703975]
 [-0.275159  ]
 [ 0.30163756]
 [ 0.00189658]
 [-0.04129073]
 [-0.7482023 ]
 [-0.7198128 ]
 [-0.85978866]]


In [49]:
#Usage of Placeholders
tf.reset_default_graph()
A=tf.placeholder(tf.float32,shape=(1,3),name="A")
B=tf.placeholder(tf.float32,shape=(1,3),name="B")
C=A+B

with tf.Session() as sess:
    c_val=C.eval(feed_dict={A:[[1,2,3]],B:[[1,2,3]]})#Give only one value in feed dictionary
    print(c_val)

[[2. 4. 6.]]


In [52]:
#Mini batch gradient descent using placeholders and splitting the training data into small sets
tf.reset_default_graph()

import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

learning_rate=0.01
n_epochs=1000
batch_size=100

def feed_next_batch(x,y,batch_size,batch_index,epoch,n_batches):
    m,n=x.shape
    np.random.seed(epoch*n_batches+batch_index)
    indices=np.random.randint(m,size=batch_size)
    return x[indices],y[indices]

housing=fetch_california_housing()
m,n=housing.data.shape

n_batches=int(np.ceil(m/batch_size))

housing_data_plus_bias=np.c_[housing.data,np.ones((m,1))] #adding bias feature as 1 for all entities

scaler=StandardScaler()
housing_data_scaled=scaler.fit_transform(housing_data_plus_bias)


x=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="Y")

theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),dtype=tf.float32,name="theta")
y_pred=tf.matmul(x,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch,y_batch=feed_next_batch(housing_data_scaled,housing.target.reshape(-1,1),batch_size,batch_index,epoch,n_batches)
            sess.run(training_op,feed_dict={x:x_batch,y:y_batch})
    print(theta.eval())

[[ 0.83778447]
 [ 0.10645497]
 [-0.25947905]
 [ 0.29196447]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.8903469 ]
 [-0.8524218 ]
 [ 0.42640734]]


In [56]:
#Saving the model using saver
tf.reset_default_graph()

import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

learning_rate=0.01
n_epochs=1000
batch_size=100

def feed_next_batch(x,y,batch_size,batch_index,epoch,n_batches):
    m,n=x.shape
    np.random.seed(epoch*n_batches+batch_index)
    indices=np.random.randint(m,size=batch_size)
    return x[indices],y[indices]

housing=fetch_california_housing()
m,n=housing.data.shape

n_batches=int(np.ceil(m/batch_size))

housing_data_plus_bias=np.c_[housing.data,np.ones((m,1))] #adding bias feature as 1 for all entities

scaler=StandardScaler()
housing_data_scaled=scaler.fit_transform(housing_data_plus_bias)


x=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="Y")

theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),dtype=tf.float32,name="theta")
y_pred=tf.matmul(x,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()
saver=tf.train.Saver()# Saver initialization

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch,y_batch=feed_next_batch(housing_data_scaled,housing.target.reshape(-1,1),batch_size,batch_index,epoch,n_batches)
            sess.run(training_op,feed_dict={x:x_batch,y:y_batch})
        if(epoch%100==0):
            saver.save(sess,"/tmp/saved_model.ckpt")#Saver saving the model
    print(theta.eval())
    saver.save(sess,"/tmp/saved_model.ckpt")#Saver saving the model

with tf.Session() as sess:
    saver.restore(sess,"/tmp/saved_model.ckpt")
    print(theta.eval())

[[ 0.83778447]
 [ 0.10645497]
 [-0.25947905]
 [ 0.29196447]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.8903469 ]
 [-0.8524218 ]
 [-0.4506507 ]]
INFO:tensorflow:Restoring parameters from /tmp/saved_model.ckpt
[[ 0.83778447]
 [ 0.10645497]
 [-0.25947905]
 [ 0.29196447]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.8903469 ]
 [-0.8524218 ]
 [-0.4506507 ]]


In [61]:
#Tensorboard 
#Mini batch gradient descent using placeholders and splitting the training data into small sets
tf.reset_default_graph()

import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

learning_rate=0.01
n_epochs=1000
batch_size=100

#initializing randomnamed logs directory as it would rewrite previous changes if same directory is given
from datetime import datetime

timestamp=datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir="tf_logs/run-{}/".format(timestamp)

def feed_next_batch(x,y,batch_size,batch_index,epoch,n_batches):
    m,n=x.shape
    np.random.seed(epoch*n_batches+batch_index)
    indices=np.random.randint(m,size=batch_size)
    return x[indices],y[indices]

housing=fetch_california_housing()
m,n=housing.data.shape

n_batches=int(np.ceil(m/batch_size))

housing_data_plus_bias=np.c_[housing.data,np.ones((m,1))] #adding bias feature as 1 for all entities

scaler=StandardScaler()
housing_data_scaled=scaler.fit_transform(housing_data_plus_bias)


x=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="Y")

theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),dtype=tf.float32,name="theta")
y_pred=tf.matmul(x,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())
mse_summary=tf.summary.scalar("MSE",mse)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch,y_batch=feed_next_batch(housing_data_scaled,housing.target.reshape(-1,1),batch_size,batch_index,epoch,n_batches)
            sess.run(training_op,feed_dict={x:x_batch,y:y_batch})
        if(epoch%100==0):
            summary_str=mse_summary.eval(feed_dict={x:housing_data_scaled,y:housing.target.reshape(-1,1)})
            step=epoch%100
            file_writer.add_summary(summary_str,step)
    print(theta.eval())
file_writer.close()

[[ 0.83778447]
 [ 0.10645497]
 [-0.25947905]
 [ 0.29196447]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.8903469 ]
 [-0.8524218 ]
 [ 0.64220357]]


In [62]:
#Name Scopes
tf.reset_default_graph()

with tf.name_scope("scope_name") as scope:
    var=tf.Variable(5,name="var")
print(var.op.name)

scope_name/var


In [82]:
#Modularity with Name Scope
tf.reset_default_graph()
from datetime import datetime

timestamp=datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir="tf_logs/run-{}/".format(timestamp)


def relu(X):
    with tf.name_scope("relu") as scope:
        m,n=X.get_shape()
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal((int(n),1)),name="weights")
        bias=tf.Variable(0.0,name="bias")
        z=tf.add(tf.matmul(X,w),bias,name="z")
        relu=tf.maximum(0.0,z,name="relu")
        return relu
    
n_features=3
x=tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus=[relu(x) for i in range(3)]
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())
output=tf.add_n(relus,name="output")

In [87]:
#Sharing Variables
#Use tf.AUTO_REUSE and this takes care of the reusing of the variable without having to set strictly to True or False
tf.reset_default_graph()
from datetime import datetime

timestamp=datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir="tf_logs/run-{}/".format(timestamp)


def relu(X):
    with tf.name_scope("relu") as scope:
        with tf.variable_scope("shared",reuse=tf.AUTO_REUSE) as scope:
            threshold=tf.get_variable("threshold")
        m,n=X.get_shape()
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal((int(n),1)),name="weights")
        bias=tf.Variable(0.0,name="bias")
        z=tf.add(tf.matmul(X,w),bias,name="z")
        relu=tf.maximum(threshold,z,name="relu")
        return relu
    
n_features=3
x=tf.placeholder(tf.float32,shape=(None,n_features),name="X")
with tf.variable_scope("shared",reuse=tf.AUTO_REUSE) as scope:
            threshold=tf.get_variable("threshold",shape=(),initializer=tf.constant_initializer(0.0))
relus=[relu(x) for i in range(3)]
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())
output=tf.add_n(relus,name="output")